博客:[评价指标AUC原理及实践](https://blog.csdn.net/jiangjiang_jian/article/details/80674304)

TP（true positive）：表示样本的真实类别为正，最后预测得到的结果也为正；
FP（false positive）：表示样本的真实类别为负，最后预测得到的结果却为正；
FN（false negative）：表示样本的真实类别为正，最后预测得到的结果却为负；
TN（true negative）：表示样本的真实类别为负，最后预测得到的结果也为负.

# 准确率和召回率：
$$Precision = \frac{TP}{TP+FP}$$
$$Precision = \frac{TP}{TP+FN}$$

在许多分类学习器中，产生的是一个概率预测值，然后将这个概率预测值与一个提前设定好的分类阈值进行比较，大于该阈值则认为是正例，小于该阈值则认为是负例。如果对所有的排序结果按照概率值进行降序排序，那么阈值可以将结果截断为两部分，前面的认为是正例，后面的认为是负例。

我们可以根据实际任务的需要选取不同的阈值。如果重视精确率，我们可以设定一个很高的阈值，如果更重视召回率，可以设定一个很低的阈值。

# ROC
到这里，我们会抛出两个问题：
1)设定阈值然后再来计算精确率，召回率和F1-Score太麻烦了，这个阈值到底该设定为多少呢？有没有可以不设定阈值来直接评价我们的模型性能的方法呢？

2)排序结果很重要呀，不管预测值是多少，只要正例的预测概率都大于负例的就好了呀。

没错，ROC和AUC便可以解决我们上面抛出的两个问题。

ROC全称是“受试者工作特征”，（receiver operating characteristic）。我们根据学习器的预测结果进行排序，然后按此顺序逐个把样本作为正例进行预测，每次计算出两个重要的值，分别以这两个值作为横纵坐标作图，就得到了ROC曲线。

这两个指标是什么呢？是精确率和召回率么？并不是的，哈哈。

ROC曲线的横轴为“假正例率”（False Positive Rate,FPR)，又称为“假阳率”；纵轴为“真正例率”(True Positive Rate,TPR)，又称为“真阳率”，

假阳率，简单通俗来理解就是预测为正样本但是预测错了的可能性，显然，我们不希望该指标太高。
$$TPR = \frac{FP}{FP+TN}$$
$$TPR = \frac{TP}{TP+FN}$$

ROC计算过程如下：

1)首先每个样本都需要有一个label值，并且还需要一个预测的score值（取值0到1）;

2)然后按这个score对样本由大到小进行排序，假设这些数据位于表格中的一列，从上到下依次降序;

3)现在从上到下按照样本点的取值进行划分，位于分界点上面的我们把它归为预测为正样本，位于分界点下面的归为负样本;

4)分别计算出此时的TPR和FPR，然后在图中绘制（FPR, TPR）点。
![image.png](pic/ROC.gif)

# AUC
AUC（area under the curve）就是ROC曲线下方的面积，如下图所示，阴影部分面积即为AUC的值：
![image.png](pic/AUC.png)
AUC量化了ROC曲线表达的分类能力。这种分类能力是与概率、阈值紧密相关的，分类能力越好（AUC越大），那么输出概率越合理，排序的结果越合理。

在CTR预估中，我们不仅希望分类器给出是否点击的分类信息，更需要分类器给出准确的概率值，作为排序的依据。


## AUC 计算
### 积分

In [ ]:
auc = 0.0
height = 0.0
 
for each training example x_i, y_i：
  if y_i = 1.0:
    height = height + 1/(tp+fn)
  else 
    auc +=  height * 1/(tn+fp)
 
return auc

### Wilcoxon-Mann-Witney Test
关于AUC还有一个很有趣的性质，它和Wilcoxon-Mann-Witney是等价的，而Wilcoxon-Mann-Witney Test就是测试任意给一个正类样本和一个负类样本，正类样本的score有多大的概率大于负类样本的score。

根据这个定义我们可以来探讨一下二者为什么是等价的？首先我们偷换一下概念，其实意思还是一样的，任意给定一个负样本，所有正样本的score中有多大比例是大于该负类样本的score？ 由于每个负类样本的选中概率相同，那么Wilcoxon-Mann-Witney Test其实就是上面n2（负样本的个数）个比例的平均值。

那么对每个负样本来说，有多少的正样本的score比它的score大呢？是不是就是当结果按照score排序，阈值恰好为该负样本score时的真正例率TPR？没错，相信你的眼睛，是这样的！理解到这一层，二者等价的关系也就豁然开朗了。ROC曲线下的面积或者说AUC的值 与 测试任意给一个正类样本和一个负类样本，正类样本的score有多大的概率大于负类样本的score

哈哈，那么我们只要计算出这个概率值就好了呀。我们知道，在有限样本中我们常用的得到概率的办法就是通过频率来估计之。这种估计随着样本规模的扩大而逐渐逼近真实值。样本数越多，计算的AUC越准确类似，也和计算积分的时候，小区间划分的越细，计算的越准确是同样的道理。具体来说就是： 统计一下所有的 M×N(M为正类样本的数目，N为负类样本的数目)个正负样本对中，有多少个组中的正样本的score大于负样本的score。当二元组中正负样本的 score相等的时候，按照0.5计算。然后除以MN。公式表示如下：
$$AUC = \frac{\sum_i^n(pos_score > neg_score)+ 0.5*\sum_i^n(pos_score = neg_score)}{M*N}$$

实现这个方法的复杂度为O(n^2 )。n为样本数(即n=M+N)

### Wilcoxon-Mann-Witney Test的化简
该方法和上述第二种方法原理一样，但复杂度降低了。首先对score从大到小排序，然后令最大score对应的sample的rank值为n，第二大score对应sample的rank值为n-1，以此类推从n到1。然后把所有的正类样本的rank相加，再减去正类样本的score为最小的那M个值的情况。得到的结果就是有多少对正类样本的score值大于负类样本的score值，最后再除以M×N即可。值得注意的是，当存在score相等的时候，对于score相等的样本，需要赋予相同的rank值(无论这个相等的score是出现在同类样本还是不同类的样本之间，都需要这样处理)。具体操作就是再把所有这些score相等的样本 的rank取平均。然后再使用上述公式。此公式描述如下：
$$AUC = \frac{\sum_{ins_i\in pos}rank_{ins_i}-\frac{M*(M+1)}{2}}{M*N}$$

In [4]:
import numpy as np
 
label_all = np.random.randint(0,2,[10,1]).tolist()
pred_all = np.random.random((10,1)).tolist()
 
print(label_all)
print(pred_all)
 
posNum = len(list(filter(lambda s: s[0] == 1, label_all)))
 
if (posNum > 0):
    negNum = len(label_all) - posNum
    sortedq = sorted(enumerate(pred_all), key=lambda x: x[1])
    print(sortedq)
 
    posRankSum = 0
    for j in range(len(pred_all)):
        if (label_all[j][0] == 1):
            posRankSum += list(map(lambda x: x[0], sortedq)).index(j) + 1
    auc = (posRankSum - posNum * (posNum + 1) / 2) / (posNum * negNum)
    print("auc:", auc)


[[1], [1], [1], [1], [0], [1], [1], [0], [0], [0]]
[[0.10708416475212068], [0.6731169312010814], [0.8365703548176777], [0.4083698112189523], [0.8247175559009063], [0.7652274659294158], [0.9108530046952538], [0.605243715011975], [0.9415109252745493], [0.6590666639267828]]
[(0, [0.10708416475212068]), (3, [0.4083698112189523]), (7, [0.605243715011975]), (9, [0.6590666639267828]), (1, [0.6731169312010814]), (5, [0.7652274659294158]), (4, [0.8247175559009063]), (2, [0.8365703548176777]), (6, [0.9108530046952538]), (8, [0.9415109252745493])]
auc: 0.4166666666666667
